In [1]:
import pandas as pd 

from sklearn.datasets import load_iris as load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from matplotlib.pyplot import plot, show

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [2]:
# train = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
# test = pd.read_csv('./house-prices-advanced-regression-techniques/test.csv')
from google.colab import files
uploaded = files.upload()
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

Saving test.csv to test.csv
Saving train.csv to train.csv


In [3]:
class feature_eng(dict):
    def __init__(self, data):
        self.data = data
        
    def show_columns_with_null(self):
        lst , counter = [] , 1
        for i in self.data.columns:
            if data[i].isna().sum():
                lst.append(i)
                print(f'{counter}. {i} - {self.data[i].dtype} : {self.data[i].isna().sum()} nulls')
                print('----------------------------------------')
                counter += 1
        print(f'total columns with null: {len(lst)}')
        return lst
    
    def show_columns_are_ojebct(self):
        lst , counter = [] , 1
        for i in self.data.columns:
            if self.data[i].dtype == 'object':
                lst.append(i)
                print(f'{counter}.  {i} - {self.data[i].dtype} ')
                print(f"     number of nulls: {self.data[i].isna().sum()} , number of elements: {len(self.data[i].value_counts())}")
                print('----------------------------------------')
                counter += 1
        print(f'tatol object columns : {len(lst)}')
        return lst
    
    def fill_null(self,field):
        if self.data[field].dtype == 'object':
            self.data[field].fillna('*',inplace=True)
        elif input(f"field type is {self.data[field].dtype}, do u want to fill with median?") == "yes":
            self.data[field].fillna(self.data[field].median(), inplace=True) 
            print(f'median of {field} column is {self.data[field].median()}')
        else:
            return self.data
            
    # def make_one_hot(self, field):
    #     temp = pd.get_dummies(self.data[field], prefix=field)
    #     self.data.drop(field, axis=1, inplace=True, errors='ignore')   
    #     return pd.concat([self.data, temp], axis=1)
    
    def labelencode(self,field):
        if self.data[field].dtype != 'object':
            print(f'type of this field is {self.data[field].dtype}')
            return self.data
        print(self.data[field].value_counts())
        print(f'number of titles : {len(self.data[field].value_counts())}')
        self.data[field] = LabelEncoder().fit_transform(data.Street) if input('do you want to proceed?') == 'yes' else self.data[field] 
        return self.data
    
    def labelencode_ordering(self,field):
        pass
    
    def impute(self):
        self.data = IterativeImputer().fit_transform(self.data)
        return self.data
        

In [46]:
# train.describe()
# train.info()
# test.info()
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [47]:
y_train = train['SalePrice']
id_test = test['Id']
x_train = train.drop(['SalePrice'], axis=1, errors='ignore', inplace=False)
data_row = pd.concat([x_train, test])
data = data_row.drop('Id', axis=1, errors='ignore', inplace=False)
data.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [48]:
ins = feature_eng(data)
object_list = ins.show_columns_are_ojebct()

1.  MSZoning - object 
     number of nulls: 4 , number of elements: 5
----------------------------------------
2.  Street - object 
     number of nulls: 0 , number of elements: 2
----------------------------------------
3.  Alley - object 
     number of nulls: 2721 , number of elements: 2
----------------------------------------
4.  LotShape - object 
     number of nulls: 0 , number of elements: 4
----------------------------------------
5.  LandContour - object 
     number of nulls: 0 , number of elements: 4
----------------------------------------
6.  Utilities - object 
     number of nulls: 2 , number of elements: 2
----------------------------------------
7.  LotConfig - object 
     number of nulls: 0 , number of elements: 5
----------------------------------------
8.  LandSlope - object 
     number of nulls: 0 , number of elements: 3
----------------------------------------
9.  Neighborhood - object 
     number of nulls: 0 , number of elements: 25
------------------------

In [49]:
for i in object_list:
    ins.fill_null(i)
ins.data.head(10)
ins.data.shape

(2919, 79)

In [50]:
def make_one_hot(data, field):
    temp = pd.get_dummies(data[field], prefix=field)
    data.drop(field, axis=1, inplace=True)
    return pd.concat([data, temp], axis=1)

In [51]:
for i in object_list:
  data = make_one_hot(data, i)

In [52]:
data.shape

(2919, 311)

In [53]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

data = IterativeImputer().fit_transform(data)

In [70]:
# scale(data)
x_train, x_test = data[:len(x_train)], data[len(x_train):]
x_train_scaled = scale(x_train)
x_train.shape

(1460, 311)

In [106]:
model = Sequential()
rows, cols = data.shape
model.add(Dense(1, input_dim=cols))
model.compile(loss='mse', optimizer=SGD(lr=0.001))

In [111]:
x_train_i, x_test_i , y_train_i, y_test_i = train_test_split(x_train_scaled, y_train, test_size=0.2, shuffle=True)
x_val, x_ts, y_val, y_ts = train_test_split(x_test_i, y_test_i, test_size=0.5, shuffle=True) 
print(y_val.shape, x_ts.shape, x_train_i.shape)
history_2 = model.fit(x_train_i, y_train_i, epochs=50)
print(mean_absolute_error(model.predict(x_val), y_val))
print(mean_absolute_error(model.predict(x_ts), y_ts))

(146,) (146, 311) (1168, 311)
Epoch 1/50
37/37 [==============================] - 0s 939us/step - loss: 455210880.0000
Epoch 2/50
37/37 [==============================] - 0s 984us/step - loss: 452458688.0000
Epoch 3/50
37/37 [==============================] - 0s 957us/step - loss: 450432000.0000
Epoch 4/50
37/37 [==============================] - 0s 965us/step - loss: 448820704.0000
Epoch 5/50
37/37 [==============================] - 0s 869us/step - loss: 447178016.0000
Epoch 6/50
37/37 [==============================] - 0s 873us/step - loss: 444879232.0000
Epoch 7/50
37/37 [==============================] - 0s 932us/step - loss: 445112896.0000
Epoch 8/50
37/37 [==============================] - 0s 887us/step - loss: 443152704.0000
Epoch 9/50
37/37 [==============================] - 0s 875us/step - loss: 442846656.0000
Epoch 10/50
37/37 [==============================] - 0s 891us/step - loss: 441034656.0000
Epoch 11/50
37/37 [==============================] - 0s 946us/step - loss: 4406

In [110]:
history = model.fit(x_train_scaled,y_train, epochs=50)
predictions = model.predict(x_test)
new = predictions.reshape(-1)
print(new.shape)
df = pd.DataFrame(data=list(zip(id_test, new)), columns=['Id', 'SalePrice'])

from google.colab import files
df.to_csv('submis.csv', index=False) 
files.download('submis.csv')
df.head(2)

Epoch 1/50
46/46 [==============================] - 0s 1ms/step - loss: 451326400.0000
Epoch 2/50
46/46 [==============================] - 0s 1ms/step - loss: 447303008.0000
Epoch 3/50
46/46 [==============================] - 0s 1ms/step - loss: 446095968.0000
Epoch 4/50
46/46 [==============================] - 0s 1ms/step - loss: 444442528.0000
Epoch 5/50
46/46 [==============================] - 0s 968us/step - loss: 443392736.0000
Epoch 6/50
46/46 [==============================] - 0s 883us/step - loss: 443070048.0000
Epoch 7/50
46/46 [==============================] - 0s 839us/step - loss: 441893056.0000
Epoch 8/50
46/46 [==============================] - 0s 990us/step - loss: 440995392.0000
Epoch 9/50
46/46 [==============================] - 0s 899us/step - loss: 440578912.0000
Epoch 10/50
46/46 [==============================] - 0s 883us/step - loss: 440815968.0000
Epoch 11/50
46/46 [==============================] - 0s 971us/step - loss: 439900160.0000
Epoch 12/50
46/46 [========

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Id,SalePrice
0,1461,125716592.0
1,1462,162715056.0
